In [4]:
import pandas as pd
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Dropout, BatchNormalization
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from tensorflow.keras.callbacks import EarlyStopping

# Load the Diagnostics.xlsx data
diagnostics_file = "../../../../Datasets/12-lead electrocardiogram database/Diagnostics.xlsx"
diagnostics_df = pd.read_excel(diagnostics_file)

# Rename "SA" to "SI" in the "Rhythm" column
diagnostics_df["Rhythm"] = diagnostics_df["Rhythm"].replace("SA", "SI")

# Drop rows with any missing values
diagnostics_df = diagnostics_df.dropna()

# Encode "Gender" column: 0 for "MALE" and 1 for "FEMALE"
diagnostics_df["Gender"] = diagnostics_df["Gender"].map({"MALE": 0, "FEMALE": 1})

# Merge specified labels
merge_mapping = {
    "AF": "AFIB",
    "AFIB": "AFIB",
    "SVT": "GSVT",
    "AT": "GSVT",
    "SAAWR": "GSVT",
    "ST": "GSVT",
    "AVNRT": "GSVT",
    "AVRT": "GSVT",
    "SB": "SB",
    "SR": "SR",
    "SI": "SR"
}
diagnostics_df["Rhythm"] = diagnostics_df["Rhythm"].map(merge_mapping)

# Separate features and labels
features = diagnostics_df.drop(columns=["FileName", "Rhythm", "Beat"]).values
labels = diagnostics_df["Rhythm"].values  # Using "Rhythm" as the target variable

# Convert features to float32
features = features.astype("float32")

# Encode labels as one-hot with merged classes
unique_labels = np.unique(labels)
label_map = {label: index for index, label in enumerate(unique_labels)}
labels_encoded = to_categorical([label_map[label] for label in labels])

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(features, labels_encoded, test_size=0.2, random_state=42)

In [5]:
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(8516, 13) (8516, 4)
(2130, 13) (2130, 4)


In [6]:
mlp = Sequential([
    Dense(32, activation="relu", input_shape=(X_train.shape[1],)),
    BatchNormalization(),
    Dense(16, activation="relu", input_shape=(X_train.shape[1],)),
    BatchNormalization(),
    Dense(labels_encoded.shape[1], activation="softmax")
])

mlp.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

mlp.fit(X_train, y_train, epochs=500, batch_size=64, validation_split=0.2,
        # callbacks=[early_stopping]
        )

# Evaluate the model
y_pred = mlp.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_test_classes = np.argmax(y_test, axis=1)

# Map back to original labels for a readable report
label_names = [label for label, index in sorted(label_map.items(), key=lambda item: item[1])]
print("\nClassification Report:\n")
print(classification_report(y_test_classes, y_pred_classes, target_names=label_names, digits=5))


Epoch 1/500


/home/denuvo-drm/miniconda3/envs/CompositeADLRecognition/lib/python3.10/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


107/107 ━━━━━━━━━━━━━━━━━━━━ 3s 15ms/step - accuracy: 0.4912 - loss: 1.3200 - val_accuracy: 0.4624 - val_loss: 1.3216
Epoch 2/500
107/107 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7741 - loss: 0.6428 - val_accuracy: 0.7823 - val_loss: 0.6316
Epoch 3/500
107/107 ━━━━━━━━━━━━━━━━━━━━ 0s 794us/step - accuracy: 0.8293 - loss: 0.4963 - val_accuracy: 0.7418 - val_loss: 0.6902
Epoch 4/500
107/107 ━━━━━━━━━━━━━━━━━━━━ 0s 815us/step - accuracy: 0.8324 - loss: 0.4530 - val_accuracy: 0.8322 - val_loss: 0.4507
Epoch 5/500
107/107 ━━━━━━━━━━━━━━━━━━━━ 0s 806us/step - accuracy: 0.8219 - loss: 0.4583 - val_accuracy: 0.7999 - val_loss: 0.5414
Epoch 6/500
107/107 ━━━━━━━━━━━━━━━━━━━━ 0s 793us/step - accuracy: 0.8393 - loss: 0.4250 - val_accuracy: 0.8298 - val_loss: 0.4912
Epoch 7/500
107/107 ━━━━━━━━━━━━━━━━━━━━ 0s 799us/step - accuracy: 0.8441 - loss: 0.4110 - val_accuracy: 0.8163 - val_loss: 0.5036
Epoch 8/500
107/107 ━━━━━━━━━━━━━━━━━━━━ 0s 801us/step - accuracy: 0.8410 - loss: 0.4177 - val_acc

In [7]:
# Reshape the data for CNN input: we will use 2D convolution, so we need to reshape (samples, features) into (samples, time_steps, channels)
# Here, each ECG feature (e.g., Ventricular Rate, Atrial Rate) will be treated as a feature in the 2D matrix, and time is along one axis.
X_train_reshaped = X_train.reshape(X_train.shape[0], X_train.shape[1], 1, 1)  # (samples, features, 1, 1)
X_test_reshaped = X_test.reshape(X_test.shape[0], X_test.shape[1], 1, 1)  # (samples, features, 1, 1)

# Define the CNN+MLP hybrid model with 2D convolution
model = Sequential([
    # 2D Convolutional layer for feature extraction
    Conv2D(64, (3, 1), activation='relu', input_shape=(X_train_reshaped.shape[1], 1, 1)),
    MaxPooling2D((2, 1)),
    Dropout(0.3),

    # Flatten the output from CNN layer and pass it to dense layers
    Flatten(),

    # MLP layers for classification
    Dense(128, activation="relu"),
    Dropout(0.3),
    Dense(64, activation="relu"),
    Dense(labels_encoded.shape[1], activation="softmax")  # Output layer with the number of merged classes
])

# Compile the model
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

# Set up early stopping
# early_stopping = EarlyStopping(monitor="val_loss", patience=5, restore_best_weights=True)

# Train the model
model.fit(X_train_reshaped, y_train, validation_data=(X_test_reshaped, y_test), epochs=500, batch_size=64,
          # callbacks=[early_stopping]
          )

# Evaluate the model
y_pred = model.predict(X_test_reshaped)
y_pred_classes = np.argmax(y_pred, axis=1)
y_test_classes = np.argmax(y_test, axis=1)

# Map back to original labels for a readable report
label_names = [label for label, index in sorted(label_map.items(), key=lambda item: item[1])]
print("\nClassification Report:\n")
print(classification_report(y_test_classes, y_pred_classes, target_names=label_names, digits=5))

Epoch 1/500


/home/denuvo-drm/miniconda3/envs/CompositeADLRecognition/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


134/134 ━━━━━━━━━━━━━━━━━━━━ 5s 21ms/step - accuracy: 0.3579 - loss: 13.0508 - val_accuracy: 0.5498 - val_loss: 1.0531
Epoch 2/500
134/134 ━━━━━━━━━━━━━━━━━━━━ 0s 858us/step - accuracy: 0.4745 - loss: 1.2916 - val_accuracy: 0.6437 - val_loss: 0.8412
Epoch 3/500
134/134 ━━━━━━━━━━━━━━━━━━━━ 0s 901us/step - accuracy: 0.5547 - loss: 1.0056 - val_accuracy: 0.6934 - val_loss: 0.8290
Epoch 4/500
134/134 ━━━━━━━━━━━━━━━━━━━━ 0s 846us/step - accuracy: 0.5691 - loss: 0.9291 - val_accuracy: 0.6577 - val_loss: 0.7794
Epoch 5/500
134/134 ━━━━━━━━━━━━━━━━━━━━ 0s 903us/step - accuracy: 0.6028 - loss: 0.8777 - val_accuracy: 0.6911 - val_loss: 0.7342
Epoch 6/500
134/134 ━━━━━━━━━━━━━━━━━━━━ 0s 810us/step - accuracy: 0.6334 - loss: 0.8128 - val_accuracy: 0.7690 - val_loss: 0.6747
Epoch 7/500
134/134 ━━━━━━━━━━━━━━━━━━━━ 0s 815us/step - accuracy: 0.6435 - loss: 0.7955 - val_accuracy: 0.6423 - val_loss: 0.6837
Epoch 8/500
134/134 ━━━━━━━━━━━━━━━━━━━━ 0s 910us/step - accuracy: 0.6575 - loss: 0.7841 - val_

In [8]:
from sklearn.tree import DecisionTreeClassifier

# Decision Tree
dt = DecisionTreeClassifier(random_state=43)
dt.fit(X_train.reshape(X_train.shape[0], -1), y_train)
y_pred_dt = dt.predict(X_test.reshape(X_test.shape[0], -1))
print("Decision Tree Classification Report:")
print(classification_report(y_test, y_pred_dt, digits=5, target_names=label_names))
print("Depth of the tree", dt.get_depth())
print("Leaf nodes of the tree", dt.get_n_leaves())

Decision Tree Classification Report:
              precision    recall  f1-score   support

        AFIB    0.68613   0.66509   0.67545       424
        GSVT    0.79878   0.81535   0.80698       482
          SB    0.97570   0.98198   0.97883       777
          SR    0.87865   0.87472   0.87668       447

   micro avg    0.85869   0.85869   0.85869      2130
   macro avg    0.83482   0.83429   0.83449      2130
weighted avg    0.85766   0.85869   0.85812      2130
 samples avg    0.85869   0.85869   0.85869      2130

Depth of the tree 24
Leaf nodes of the tree 688
